# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [9]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
data_json = r.json()
df = pd.DataFrame(data_json['items'])
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,8050,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.000703/2010-61,2011-04-29T03:00:00Z,Operação contratual interna,PNAFM,Instituição Financeira Nacional,Caixa Econômica Federal,Real,49000000.00,1,0,06/06/2011
1,5914,Município,Nilópolis,3303203,RJ,None,Regularizado,17944.000231/2012-16,2012-08-20T03:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,4178061.55,0,0,05/09/2012
2,5643,Município,São João de Meriti,3305109,RJ,None,Arquivado,17944.000161/2010-26,2010-10-15T03:00:00Z,Operação contratual interna,PAC - Saneamento - 2009 A,Instituição Financeira Nacional,Caixa Econômica Federal,Real,50074939.06,0,0,20/12/2010
3,38893,Município,Paty do Alferes,3303856,RJ,PVL02.007600/2019-05,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,3512000.00,0,1,30/10/2019
4,56075,Município,Resende,3304201,RJ,PVL02.003504/2022-85,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,2160000.00,0,1,28/07/2022


In [16]:
# 2) Seu código aqui
ranking = df.status.value_counts()
ranking[0:3]

Deferido        94
Arquivado       53
Regularizado    25
Name: status, dtype: int64

In [89]:
# 3) Seu código aqui
df['ano_status'] = df['data_status'].str.split('/').apply(lambda x: x[-1])
df.ano_status.value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2012    13
2022    12
2011    10
2009     9
2015     8
2017     5
2021     5
2002     4
2023     4
2018     4
2016     4
2006     4
2003     1
2004     1
Name: ano_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [185]:
#1) Seu código aqui
url2 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
r2 = requests.get(url2)
data_json2 = r2.json()
df2 = pd.DataFrame(data_json2['items'])

def consulta(estado: 'str', tipo_interessado: 'str'):
    estado = estado.upper()
    tipo_interessado = tipo_interessado.upper()
    
    if tipo_interessado == 'MUNICIPIO':
        df3 = df2[df2['tipo_interessado'] == 'Município']
        if estado == 'ACRE':
            df4 = df3[df3['uf'] == 'AC']
        elif estado == 'ALAGOAS':
            df4 = df3[df3['uf'] == 'AL']
        elif estado == 'AMAPA':
            df4 = df3[df3['uf'] == 'AP']
        elif estado == 'AMAZONAS':
            df4 = df3[df3['uf'] == 'AM']
        elif estado == 'BAHIA':
            df4 = df3[df3['uf'] == 'BA']
        elif estado == 'CEARA':
            df4 = df3[df3['uf'] == 'CE']
        elif estado == 'ESPIRITO SANTO':
            df4 = df3[df3['uf'] == 'ES']
        elif estado == 'GOIAS':
            df4 = df3[df3['uf'] == 'GO']
        elif estado == 'MARANHAO':
            df4 = df3[df3['uf'] == 'MA']
        elif estado == 'MATO GROSSO':
            df4 = df3[df3['uf'] == 'MT']
        elif estado == 'MATO GROSSO DO SUL':
            df4 = df3[df3['uf'] == 'MS']
        elif estado == 'MINAS GERAIS':
            df4 = df3[df3['uf'] == 'MG']
        elif estado == 'PARA':
            df4 = df3[df3['uf'] == 'PA']
        elif estado == 'PARAIBA':
            df4 = df3[df3['uf'] == 'PB']
        elif estado == 'PARANA':
            df4 = df3[df3['uf'] == 'PR']
        elif estado == 'PERNAMBUCO':
            df4 = df3[df3['uf'] == 'PE']
        elif estado == 'PIAUI':
            df4 = df3[df3['uf'] == 'PI']
        elif estado == 'RIO DE JANEIRO':
            df4 = df3[df3['uf'] == 'RJ']
        elif estado == 'RIO GRANDE DO NORTE':
            df4 = df3[df3['uf'] == 'RN']
        elif estado == 'RIO GRANDE DO SUL':
            df4 = df3[df3['uf'] == 'RS']
        elif estado == 'RONDONIA':
            df4 = df3[df3['uf'] == 'RO']
        elif estado == 'RORAIMA':
            df4 = df3[df3['uf'] == 'RR']
        elif estado == 'SANTA CATARINA':
            df4 = df3[df3['uf'] == 'SC']
        elif estado == 'SAO PAULO':
            df4 = df3[df3['uf'] == 'SP']
        elif estado == 'SERGIPE':
            df4 = df3[df3['uf'] == 'SE']
        elif estado == 'TOCANTINS':
            df4 = df3[df3['uf'] == 'TO']
        elif estado == 'DISTRITO FEDERAL':
            df4 = df3[df3['uf'] == 'DF']
            
    elif tipo_interessado == 'ESTADO':
        df3 = df2[df2['tipo_interessado'] == 'Estado']
        if estado == 'ACRE':
            df4 = df3[df3['uf'] == 'AC']
        elif estado == 'ALAGOAS':
            df4 = df3[df3['uf'] == 'AL']
        elif estado == 'AMAPA':
            df4 = df3[df3['uf'] == 'AP']
        elif estado == 'AMAZONAS':
            df4 = df3[df3['uf'] == 'AM']
        elif estado == 'BAHIA':
            df4 = df3[df3['uf'] == 'BA']
        elif estado == 'CEARA':
            df4 = df3[df3['uf'] == 'CE']
        elif estado == 'ESPIRITO SANTO':
            df4 = df3[df3['uf'] == 'ES']
        elif estado == 'GOIAS':
            df4 = df3[df3['uf'] == 'GO']
        elif estado == 'MARANHAO':
            df4 = df3[df3['uf'] == 'MA']
        elif estado == 'MATO GROSSO':
            df4 = df3[df3['uf'] == 'MT']
        elif estado == 'MATO GROSSO DO SUL':
            df4 = df3[df3['uf'] == 'MS']
        elif estado == 'MINAS GERAIS':
            df4 = df3[df3['uf'] == 'MG']
        elif estado == 'PARA':
            df4 = df3[df3['uf'] == 'PA']
        elif estado == 'PARAIBA':
            df4 = df3[df3['uf'] == 'PB']
        elif estado == 'PARANA':
            df4 = df3[df3['uf'] == 'PR']
        elif estado == 'PERNAMBUCO':
            df4 = df3[df3['uf'] == 'PE']
        elif estado == 'PIAUI':
            df4 = df3[df3['uf'] == 'PI']
        elif estado == 'RIO DE JANEIRO':
            df4 = df3[df3['uf'] == 'RJ']
        elif estado == 'RIO GRANDE DO NORTE':
            df4 = df3[df3['uf'] == 'RN']
        elif estado == 'RIO GRANDE DO SUL':
            df4 = df3[df3['uf'] == 'RS']
        elif estado == 'RONDONIA':
            df4 = df3[df3['uf'] == 'RO']
        elif estado == 'RORAIMA':
            df4 = df3[df3['uf'] == 'RR']
        elif estado == 'SANTA CATARINA':
            df4 = df3[df3['uf'] == 'SC']
        elif estado == 'SAO PAULO':
            df4 = df3[df3['uf'] == 'SP']
        elif estado == 'SERGIPE':
            df4 = df3[df3['uf'] == 'SE']
        elif estado == 'TOCANTINS':
            df4 = df3[df3['uf'] == 'TO']
        elif estado == 'DISTRITO FEDERAL':
            df4 = df3[df3['uf'] == 'DF']
    
    
    return df4
    

In [186]:
# 2) Seu código aqui
mg = consulta(estado = 'minas gerais', tipo_interessado = 'estado')
filtro = mg['status'] == 'Arquivado por decurso de prazo'
mg[filtro].value_counts()
#nenhum status desse tipo

Series([], dtype: int64)

In [210]:
# 3) Seu código aqui
ba = consulta(estado = 'bahia', tipo_interessado = 'municipio')
ba_def = ba.loc[ba['status'] == 'Deferido']
ranking_ba = ba_def.interessado.value_counts()
ranking_ba[:1]

Lauro de Freitas    3
Name: interessado, dtype: int64

In [218]:
# 4) Seu código aqui
ba_es = consulta(estado = 'bahia', tipo_interessado = 'estado')
ba_es.head()
ba_es.to_csv('ba_es.csv', index= False, sep=';')